# Iris
## iris:Treefarms

In [5]:
# Import the package
import pandas as pd
import numpy as np
from pydrf.model import DRFModel 
from sklearn import datasets
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from treefarms import TREEFARMS




In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [7]:
data = datasets.load_iris()
# iris_df = pd.DataFrame(data.data, columns=data.feature_names)
iris_df = pd.DataFrame(data = np.c_[data['data'], data['target']],
                       columns= data.feature_names + ['target'])
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [8]:
X, y = iris_df.iloc[:, :-1], iris_df.iloc[:, -1]
print(X.columns)
y

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)'],
      dtype='object')


0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
145    2.0
146    2.0
147    2.0
148    2.0
149    2.0
Name: target, Length: 150, dtype: float64

In [9]:
for col in X.columns:
    # 使用 qcut 分為三個等分
    X[col + '_bin'] = pd.qcut(X[col], 3, labels=False, duplicates='drop')
    # 進行 one-hot 編碼
    dummies = pd.get_dummies(X[col + '_bin'], prefix=col)
    X = pd.concat([X, dummies], axis=1)
    # 刪除原始的分箱特徵
    X.drop(col + '_bin', axis=1, inplace=True)
    X.drop(col, axis=1, inplace=True)

print(X.head())

   sepal length (cm)_0  sepal length (cm)_1  sepal length (cm)_2  \
0                    1                    0                    0   
1                    1                    0                    0   
2                    1                    0                    0   
3                    1                    0                    0   
4                    1                    0                    0   

   sepal width (cm)_0  sepal width (cm)_1  sepal width (cm)_2  \
0                   0                   0                   1   
1                   0                   1                   0   
2                   0                   1                   0   
3                   0                   1                   0   
4                   0                   0                   1   

   petal length (cm)_0  petal length (cm)_1  petal length (cm)_2  \
0                    1                    0                    0   
1                    1                    0                    0

In [10]:
# train TREEFARMS model
config = {
    "regularization": 0.1,  # regularization penalizes the tree with more leaves. We recommend to set it to relative high value to find a sparse tree.
    "rashomon_bound_multiplier": 0.1,  # rashomon bound multiplier indicates how large of a Rashomon set would you like to get
    "rashomon": True,
}

treefarms_model = TREEFARMS(config)

treefarms_model.fit(X, y)


null
Finding Optimal Objective...
treefarms reported successful execution
training completed. Number of trees in the Rashomon set: 22


{
  "false": {
    "false": {
      "complexity": 0.10000000149011612,
      "loss": 0.026666667312383652,
      "name": "target",
      "prediction": 1.0
    },
    "feature": 9,
    "name": "petal width (cm)_0",
    "reference": 1,
    "relation": "==",
    "true": {
      "complexity": 0.10000000149011612,
      "loss": 0.0,
      "name": "target",
      "prediction": 0.0
    },
    "type": "integral"
  },
  "feature": 11,
  "model_objective": 0.3400000035762787,
  "name": "petal width (cm)_2",
  "reference": 1,
  "relation": "==",
  "true": {
    "complexity": 0.10000000149011612,
    "loss": 0.013333333656191826,
    "name": "target",
    "prediction": 2.0
  },
  "type": "integral"
}
{
  "false": {
    "false": {
      "complexity": 0.10000000149011612,
      "loss": 0.013333333656191826,
      "name": "target",
      "prediction": 2.0
    },
    "feature": 6,
    "name": "petal length (cm)_0",
    "reference": 1,
    "relation": "==",
    "true": {
      "complexity": 0.100000001

In [11]:
# treefarms_model.visualize()


In [12]:
for i in range(22):
    print (treefarms_model[i].score(X,y))

0.9533333333333334
0.96
0.96
0.96
0.96
0.96
0.96
0.96
0.96
0.96
0.96
0.94
0.9466666666666667
0.9466666666666667
0.9466666666666667
0.9466666666666667
0.9466666666666667
0.9466666666666667
0.9466666666666667
0.9466666666666667
0.9466666666666667
0.9466666666666667


## iris: DRF+ Decision tree

In [13]:
# Define the structure of DRF
param_list = [
    [{"max_leaf_nodes":6, "n_estimators":100, "max_features":0.8}],
    [{"max_leaf_nodes":6, "n_estimators":100, "max_features":0.8}],
    [{"max_leaf_nodes":6, "n_estimators":100, "max_features":0.8}],
    [{"max_leaf_nodes":2, "n_estimators":100, "max_features":0.8}],
    [{"max_leaf_nodes":2, "n_estimators":100, "max_features":0.8}]
    ]

# Fit the model
drf = DRFModel(parallel_cores=5, parameter_list = param_list)
drf_model = drf.fit(X, y)


Layer 0 training...finished.
Layer 1 training...finished.
Layer 2 training...finished.
Layer 3 training...finished.
Layer 4 training...finished.


In [38]:
Decision_tree = DecisionTreeClassifier(max_depth=2)
Decision_tree.fit(X, y)
print("decision_tree predict accuarcy:", Decision_tree.score(X, y))
    

decision_tree predict accuarcy: 0.96


In [37]:
# results of 5 different layer representations
for i in range(0,6):
    transform_train = drf_model.transform_data(X.values, stop_layer = i)
    # drf predict
    outcome = drf_model.predict(X.values)
    print("drf predict accuarcy:", accuracy_score(outcome, y))
    # drf+tree predict
    Decision_tree = DecisionTreeClassifier(max_depth=2)
    Decision_tree.fit(transform_train, y)
    print("drf+decision_tree predict accuarcy:", Decision_tree.score(transform_train, y))
    

Transforming data...finished.
Transforming data...finished.
drf predict accuarcy: 0.98
drf+decision_tree predict accuarcy: 0.96
Transforming data...finished.
Transforming data...finished.


/home/yorkcho/pydrf/Code/pydrf/model.py:222: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  forest_prediction = stats.mode(trees_prediction)[0].reshape(trees_prediction.shape[1],)
/home/yorkcho/pydrf/Code/pydrf/model.py:222: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warni

drf predict accuarcy: 0.98
drf+decision_tree predict accuarcy: 0.98
Transforming data...finished.
Transforming data...finished.
drf predict accuarcy: 0.98
drf+decision_tree predict accuarcy: 0.98
Transforming data...finished.
Transforming data...

/home/yorkcho/pydrf/Code/pydrf/model.py:222: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  forest_prediction = stats.mode(trees_prediction)[0].reshape(trees_prediction.shape[1],)


finished.
drf predict accuarcy: 0.98
drf+decision_tree predict accuarcy: 0.98
Transforming data...finished.
Transforming data...

/home/yorkcho/pydrf/Code/pydrf/model.py:222: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  forest_prediction = stats.mode(trees_prediction)[0].reshape(trees_prediction.shape[1],)


finished.
drf predict accuarcy: 0.98
drf+decision_tree predict accuarcy: 0.98
Transforming data...finished.
Transforming data...

/home/yorkcho/pydrf/Code/pydrf/model.py:222: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  forest_prediction = stats.mode(trees_prediction)[0].reshape(trees_prediction.shape[1],)


finished.
drf predict accuarcy: 0.98
drf+decision_tree predict accuarcy: 0.98


/home/yorkcho/pydrf/Code/pydrf/model.py:222: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  forest_prediction = stats.mode(trees_prediction)[0].reshape(trees_prediction.shape[1],)


# car evaluation
## car evaluation : Treefarms

In [39]:
# read the dataset
df = pd.read_csv("../../treeFarms/experiments/datasets/car_evaluation/data.csv")
X, y = df.iloc[:, :-1], df.iloc[:, -1]
h = df.columns[:-1]
df

,a1_high,a1_low,a1_med,a2_high,a2_low,a2_med,a3_2,a3_3,a3_4,a4_2,a4_4,a5_big,a5_med,a6_high,a6_low,class_1
0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0
1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0
3,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0
4,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1723,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1
1724,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,1
1725,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0
1726,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1


In [40]:
# train TREEFARMS model
config = {
    "regularization": 0.05,  # regularization penalizes the tree with more leaves. We recommend to set it to relative high value to find a sparse tree.
    "rashomon_bound_multiplier": 0.1,  # rashomon bound multiplier indicates how large of a Rashomon set would you like to get
}

treefarms_model = TREEFARMS(config)

treefarms_model.fit(X, y)


null
Finding Optimal Objective...
treefarms reported successful execution
training completed. Number of trees in the Rashomon set: 2


{
  "false": {
    "false": {
      "complexity": 0.05000000074505806,
      "loss": 0.14467592537403107,
      "name": "class_1",
      "prediction": 1
    },
    "feature": 9,
    "name": "a4_2",
    "reference": 1,
    "relation": "==",
    "true": {
      "complexity": 0.05000000074505806,
      "loss": 0.0,
      "name": "class_1",
      "prediction": 0
    },
    "type": "integral"
  },
  "feature": 14,
  "model_objective": 0.29467594623565674,
  "name": "a6_low",
  "reference": 1,
  "relation": "==",
  "true": {
    "complexity": 0.05000000074505806,
    "loss": 0.0,
    "name": "class_1",
    "prediction": 0
  },
  "type": "integral"
}
{
  "false": {
    "false": {
      "complexity": 0.05000000074505806,
      "loss": 0.14467592537403107,
      "name": "class_1",
      "prediction": 1
    },
    "feature": 14,
    "name": "a6_low",
    "reference": 1,
    "relation": "==",
    "true": {
      "complexity": 0.05000000074505806,
      "loss": 0.0,
      "name": "class_1",
      

In [42]:
for i in range(2):
    print (treefarms_model[i].score(X,y))

0.8553240740740741
0.8553240740740741


In [41]:
first_tree = treefarms_model[0]
print("evaluating the first model in the Rashomon set", flush=True)

# get the results
train_acc = first_tree.score(X, y)
n_leaves = first_tree.leaves()
n_nodes = first_tree.nodes()

print("Training accuracy: {}".format(train_acc))
print("# of leaves: {}".format(n_leaves))
print(first_tree)

evaluating the first model in the Rashomon set
Training accuracy: 0.8553240740740741
# of leaves: 3
if feature_14 = true then:
    predicted Prediction: 0

else if feature_14 != true and feature_9 = true then:
    predicted Prediction: 0

else if feature_14 != true and feature_9 != true then:
    predicted Prediction: 1


## car evaluation: DRF+Decision tree

In [43]:
# Define the structure of DRF

param_list = [
    [{"max_leaf_nodes":6, "n_estimators":100, "max_features":0.8}],
    [{"max_leaf_nodes":6, "n_estimators":100, "max_features":0.8}],
    [{"max_leaf_nodes":6, "n_estimators":100, "max_features":0.8}],
    [{"max_leaf_nodes":2, "n_estimators":100, "max_features":0.8}],
    [{"max_leaf_nodes":2, "n_estimators":100, "max_features":0.8}]
    ]

# Fit the model
drf = DRFModel(parallel_cores = 5, parameter_list = param_list)
drf_model = drf.fit(X, y)


Layer 0 training...finished.
Layer 1 training...finished.
Layer 2 training...finished.
Layer 3 training...finished.
Layer 4 training...finished.


In [44]:
transform_train = drf_model.transform_data(X.values, stop_layer = 5)

Transforming data...finished.


In [45]:
# drf predict
outcome = drf_model.predict(X.values)
accuracy_score(outcome, y)

Transforming data...finished.


/home/yorkcho/pydrf/Code/pydrf/model.py:222: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  forest_prediction = stats.mode(trees_prediction)[0].reshape(trees_prediction.shape[1],)


0.9583333333333334

In [50]:
# drf+tree predict
Decision_tree = DecisionTreeClassifier(max_depth=2)
Decision_tree.fit(X, y)
Decision_tree.score(X, y)

0.8553240740740741

In [47]:
X.head()

,a1_high,a1_low,a1_med,a2_high,a2_low,a2_med,a3_2,a3_3,a3_4,a4_2,a4_4,a5_big,a5_med,a6_high,a6_low
0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1
1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0
3,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1
4,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0


In [54]:
# results of 5 different layer representations
for i in range(0,6):
    transform_train = drf_model.transform_data(X.values, stop_layer = i)
    # drf+tree predict
    Decision_tree = DecisionTreeClassifier(max_depth=2)
    Decision_tree.fit(transform_train, y)
    print(f"{i}th drf+decision_tree predict accuarcy:", Decision_tree.score(transform_train, y))
    

Transforming data...finished.
0th drf+decision_tree predict accuarcy: 0.8553240740740741
Transforming data...finished.
1th drf+decision_tree predict accuarcy: 0.90625
Transforming data...finished.
2th drf+decision_tree predict accuarcy: 0.9537037037037037
Transforming data...finished.
3th drf+decision_tree predict accuarcy: 0.9583333333333334
Transforming data...finished.
4th drf+decision_tree predict accuarcy: 0.9583333333333334
Transforming data...finished.
5th drf+decision_tree predict accuarcy: 0.9594907407407407
